## In this notebook, we called GPT 3.5 again to generate summaries for the 10K reports as a baseline for comparison. Below is the general process:
- Load in processed Item 7 data where all the tables were removed, useless contents were removes, and useful contents were re-ordered and keyed by each big theme (e.g. revenue, gross profit margin)
- For each report, load in contents under each theme and call GPT 3.5 either once or multiple times to generate summaries (since the API has maximum token limit for each call)
- Ask GPT 3.5 to generate a summary for each theme 
- Concat the summaries together theme by theme to form the summary for the Item 7 as a whole
- Calculate ROUGE scores and BERT score for GPT 3.5-generated summary comparing with the ground truth labels

In [ ]:
# mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai
!pip install rouge
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import packages
import pandas as pd
import numpy as np
import glob
import os.path
import re
import openai
from tqdm import tqdm
import math
from rouge import Rouge
from bert_score import score
import torch
import time
from openai.error import InvalidRequestError
import pickle

In [ ]:
# load processed data
file_path = "/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/working3/"

df = pd.read_pickle(os.path.join(file_path,'item7_text_v5.pkl'))

df['label'] = df['label'].apply(lambda x: re.sub('\n', ' ', x))

df[df['kept_report_length'] <= df['label_length']]

,id,label_length,label,kept_report_length,report,has_label
20,789019,1094,Microsoft is a technology company whose missio...,13,"{'Operating Income': '', 'Revenues': '', 'Liqu...",True
31,829323,1855,Inuvo is a technology company that develops an...,1792,"{'Operating Income': '', 'Revenues': '', 'Liqu...",True
32,843006,628,"Total revenue increased by $2,231,000, or 14%,...",13,"{'Operating Income': '', 'Revenues': '', 'Liqu...",True
36,860731,2230,The company provides integrated information ma...,13,"{'Operating Income': '', 'Revenues': '', 'Liqu...",True


In [ ]:
labels_to_remove = ['789019','829323','843006','860731']
df = df[~df['id'].isin(labels_to_remove)]

In [ ]:
# check unique themes
themes = []
for index, row in df.iterrows():
  for key in row['report'].keys():
    if key not in themes:
      themes.append(key)
themes

['Operating Income',
 'Revenues',
 'Liquidity',
 'Results of Operations',
 'Business Overview',
 'Not Important',
 'Debt',
 'Gross Profit Margin',
 'Interest expense',
 'Results of operations',
 'Interest Expense',
 'Business overview',
 'Not important']

In [ ]:
# input data cleaning
def combine_interest_expense(row):
    expense_1 = row['Interest expense']
    expense_2 = row['Interest Expense']
    
    # Combine the values of the two keys
    combined_expense = expense_1 + expense_2
    
    # Remove the 'Interest expense' key
    del row['Interest expense']
    
    # Update the 'Interest Expense' key with the combined value
    row['Interest Expense'] = str(combined_expense)
    # Remove the 'Not Important' key
    if 'Not Important' in row:
        del row['Not Important']
    
    return row

def combine_business_overview(row):
    bussiness_1 = row['Business Overview']
    bussiness_2 = row['Business overview']
    
    # Combine the values of the two keys
    combined_business = bussiness_1 + bussiness_2
    
    # Remove the 'Interest expense' key
    del row['Business overview']
    
    # Update the 'Interest Expense' key with the combined value
    row['Business Overview'] = str(combined_business)
    # Remove the 'Not Important' key
    
    return row

def combine_operations(row):
    operation_1 = row['Results of Operations']
    operation_2 = row['Results of operations']
    
    # Combine the values of the two keys
    combined_operation = operation_1 + operation_2
    
    # Remove the 'Interest expense' key
    del row['Results of operations']
    
    # Update the 'Interest Expense' key with the combined value
    row['Results of Operations'] = str(combined_operation)
    # Remove the 'Not Important' key
    
    return row

# Update the 'report' column using the apply function and the custom function
df['report'] = df['report'].apply(combine_interest_expense)
df['report'] = df['report'].apply(combine_business_overview)
df['report'] = df['report'].apply(combine_operations)

In [ ]:
def remove_underscore_sentences(text):
    sentences = text.split('. ')
    cleaned_sentences = [sentence for sentence in sentences if not re.search(r'_+', sentence)]
    return '. '.join(cleaned_sentences)

keys = [
    'Business Overview',
    'Results of Operations',
    'Revenues',
    'Gross Profit Margin',
    'Operating Income',
    'Interest Expense',
    'Liquidity',
    'Debt'
]

for index, row in df.iterrows():
    report = row['report']
    for key in keys:
        if key in report:
            report[key] = remove_underscore_sentences(report[key])

# If you need to overwrite the 'report' column with the cleaned data
df['report'] = df['report'].apply(lambda report: {key: remove_underscore_sentences(report[key]) for key in keys if key in report})

In [ ]:
df.head()

,id,label_length,label,kept_report_length,report,has_label
0,8670,1142,"AUTOMATIC DATA PROCESSING, INC. (“ADPI”) Auto...",3770,{'Business Overview': 'highlights from the yea...,True
1,50471,907,"Park City Group, Inc. (“PCGI”) The Company is ...",1599,{'Business Overview': 'the company is a saas p...,True
2,78749,756,"AGILYSYS, Inc. (“AI”) Agilysys has been a lead...",3348,{'Business Overview': ' agilysys has been a le...,True
3,317788,927,"Digital Turbine, Inc. (“DTI”) Digital Turbine,...",6580,{'Business Overview': 'we allocate the fair va...,True
4,320340,933,Intelligent Systems Corporation (“ISC”) ISC’s...,1757,{'Business Overview': 'our consolidated operat...,True


In [ ]:
newdf = df.copy()
for key in keys:
  newdf[key] = newdf['report'].apply(lambda x : x[key])
newdf.head()

,id,label_length,label,kept_report_length,report,has_label,Business Overview,Results of Operations,Revenues,Gross Profit Margin,Operating Income,Interest Expense,Liquidity,Debt
0,8670,1142,"AUTOMATIC DATA PROCESSING, INC. (“ADPI”) Auto...",3770,{'Business Overview': 'highlights from the yea...,True,"highlights from the year ended june 30, 2021 i...",,"for the year ended june 30, respectively: reve...",,"in addition to our u.s. gaap results, we use a...",,"as of june 30, 2021, cash and cash equivalent ...",the following table provides a summary of our ...
1,50471,907,"Park City Group, Inc. (“PCGI”) The Company is ...",1599,{'Business Overview': 'the company is a saas p...,True,"the company is a saas provider, and the parent...",,"during the fiscal year ended june 30, 2021, th...","cost of services and product support was $6,88...","to supplement our financial statements, histor...",,net cash provided by operating activities is s...,the company does not have any off-balance shee...
2,78749,756,"AGILYSYS, Inc. (“AI”) Agilysys has been a lead...",3348,{'Business Overview': ' agilysys has been a le...,True,agilysys has been a leader in hospitality sof...,fiscal 2021 compared with fiscal 2020 net reve...,"as required by the sec, we separately present ...",,,,overview our cash requirements consist primari...,we have not entered into any off-balance sheet...
3,317788,927,"Digital Turbine, Inc. (“DTI”) Digital Turbine,...",6580,{'Business Overview': 'we allocate the fair va...,True,we allocate the fair value of purchase conside...,"all discussions in this item 7, management’s d...",52,,"during the years ended march 31, 2021 and 2020...",fiscal 2021 compared to fiscal 2020 total inte...,our primary sources of liquidity have historic...,we do not have any relationships with unconsol...
4,320340,933,Intelligent Systems Corporation (“ISC”) ISC’s...,1757,{'Business Overview': 'our consolidated operat...,True,our consolidated operations consist of our cor...,the following discussion should be read in con...,– total revenue for the year ended december 31...,– total cost of revenue was 43 percent of tota...,"– for the twelve months ended december 31, 202...",,"our cash balance at december 31, 2020 was $37,...",


In [ ]:
# import private OpenAI API key
with open('/content/drive/MyDrive/openai.txt') as f:
    lines = f.readlines()
openai.api_key = lines[0]

In [ ]:
def generate_summary_with_gpt3point5(text, max_num_tokens, token_ratio):
    num_input_tokens = math.ceil(len(text.split(' ')) * token_ratio)
    num_output_tokens = int(max_num_tokens - num_input_tokens)
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages = [{"role": "user", "content": f'Summarize the following text in no more than {num_output_tokens} tokens: "{text}"'}],
    max_tokens=num_output_tokens,
    n=1,
    stop=None,
    temperature=0.7,
    )
    output = response['choices'][0]['message']['content']
    num_output_tokens = math.ceil(len(output.split(' ')) * token_ratio)
    print(f'num_input_tokens: {num_input_tokens}, ' \
          + f'num_output_tokens: {num_output_tokens}, ' \
          f'total: {num_input_tokens + num_output_tokens}')
    # return response.choices[0].text
    return output

In [ ]:
def truncate_text_for_gpt3point5(row, col, output_length):
  file_id = row['id']
  text = row[col].strip()
  # number of words in ground truth labels
  # number of words in report
  input_length = len(text.split(' '))
  # control length of generated summary per chunk
  # by the ratio between the input length and output length
  token_ratio = 1000.0 / 750.0
  input_token_length = math.ceil(input_length * token_ratio)
  if input_token_length < 32:
    return ''
  output_token_length = math.ceil(output_length * token_ratio)

  summaries = []
  max_num_tokens = 4000
  done = False
  num_api_calls = 0
  while not done:
    try:
      num_api_calls = math.ceil((input_token_length + output_token_length) / max_num_tokens)
      # truncate original text evenly for API calls
      split_word_indices = np.asarray(np.linspace(0, len(text.split(" ")), num_api_calls + 1), dtype = "int")
      # print(f"Processing file {file_id}, input length: {input_length}")
      print(f"key {col}, input token length: {input_token_length}, output token length: {output_token_length}")
      summaries = []
      # call API chunk by chunk
      for i in range(len(split_word_indices) - 1):
        # print(f"-----Processing text chunk from word {split_word_indices[i]} to word {split_word_indices[i+1]}.")
        chunk_text = " ".join(text.split(" ")[split_word_indices[i]:split_word_indices[i+1]])
        chunk_summary = generate_summary_with_gpt3point5(chunk_text, max_num_tokens, token_ratio)
        # wait due to OpenAI rate limit
        summaries.append(chunk_summary)
        time.sleep(15)
      done = True
    except InvalidRequestError as e:
      if "This model's maximum context length" not in e.user_message:
        raise Exception(e)
        done = True
      else:
        max_num_tokens -= 100
        print(f'reduce max_num_tokens by 100: {max_num_tokens}')

  # return generated summaries of all chunks
  return " ".join(summaries)

In [ ]:
def toDoc(r):
  doc = ''
  for key in keys:
    text = r[key].strip()
    if text != "":
      doc += key + '\n'
      doc += text + '\n\n'
  return doc.strip()

# ensure that the output directory exists
summary_path = "/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/working3/gpt"
if not os.path.exists(summary_path):
  os.makedirs(summary_path)

# loop thru all files
for i, row in tqdm(newdf.iterrows()):
  id = row['id']

  # if the file exists, then don't call gpt
  output_name = f'{summary_path}/{id}.txt'
  if os.path.exists(output_name): continue

  # estimate output lengths
  output_length = row['label_length']
  output_lengths = {}
  total = 0
  for key in keys:
    input_length = len(row[key].split(' '))
    output_lengths[key] = input_length * output_length
    total += input_length
  total = max(total, 1.0)

  # call gpt otherwise
  out = { 'id': id }
  for key in keys:
    print(f'working on {id}: {key} ...')
    out[key] = truncate_text_for_gpt3point5(row, key, math.floor(output_lengths[key] / total))

  # compose a summary document and write it out
  summary = toDoc(out)
  with open(output_name, 'w') as f:
    f.write(summary)
  # break

0it [00:00, ?it/s]

working on 50471: Business Overview ...
key Business Overview, input token length: 99, output token length: 56
num_input_tokens: 99, num_output_tokens: 90, total: 189
working on 50471: Results of Operations ...
working on 50471: Revenues ...
key Revenues, input token length: 770, output token length: 438
num_input_tokens: 770, num_output_tokens: 254, total: 1024
working on 50471: Gross Profit Margin ...
key Gross Profit Margin, input token length: 152, output token length: 86
num_input_tokens: 152, num_output_tokens: 132, total: 284
working on 50471: Operating Income ...
key Operating Income, input token length: 231, output token length: 131
num_input_tokens: 231, num_output_tokens: 103, total: 334
working on 50471: Interest Expense ...
working on 50471: Liquidity ...
key Liquidity, input token length: 804, output token length: 458
num_input_tokens: 804, num_output_tokens: 275, total: 1079
working on 50471: Debt ...
key Debt, input token length: 68, output token length: 39
num_input_to

2it [02:05, 62.77s/it]

working on 78749: Business Overview ...
key Business Overview, input token length: 358, output token length: 80
num_input_tokens: 358, num_output_tokens: 242, total: 600
working on 78749: Results of Operations ...
key Results of Operations, input token length: 2987, output token length: 675
num_input_tokens: 2987, num_output_tokens: 170, total: 3157
working on 78749: Revenues ...
key Revenues, input token length: 123, output token length: 27
num_input_tokens: 123, num_output_tokens: 91, total: 214
working on 78749: Gross Profit Margin ...
working on 78749: Operating Income ...
working on 78749: Interest Expense ...
working on 78749: Liquidity ...
key Liquidity, input token length: 815, output token length: 184
num_input_tokens: 815, num_output_tokens: 324, total: 1139
working on 78749: Debt ...
key Debt, input token length: 171, output token length: 38
num_input_tokens: 171, num_output_tokens: 108, total: 279


3it [03:58, 83.78s/it]

...

44it [1:45:57, 137.64s/it]

working on 1001601: Debt ...
working on 1002517: Business Overview ...
key Business Overview, input token length: 1043, output token length: 368
num_input_tokens: 1043, num_output_tokens: 228, total: 1271
working on 1002517: Results of Operations ...
key Results of Operations, input token length: 223, output token length: 79
num_input_tokens: 223, num_output_tokens: 179, total: 402
working on 1002517: Revenues ...
key Revenues, input token length: 1722, output token length: 607
reduce max_num_tokens by 100: 3900
key Revenues, input token length: 1722, output token length: 607
num_input_tokens: 1722, num_output_tokens: 270, total: 1992
working on 1002517: Gross Profit Margin ...
working on 1002517: Operating Income ...
key Operating Income, input token length: 550, output token length: 194
num_input_tokens: 550, num_output_tokens: 198, total: 748
working on 1002517: Interest Expense ...
working on 1002517: Liquidity ...
key Liquidity, input token length: 2388, output token length: 843
r

45it [1:48:37, 144.45s/it]

working on 1013462: Business Overview ...
key Business Overview, input token length: 2547, output token length: 492
num_input_tokens: 2547, num_output_tokens: 260, total: 2807
working on 1013462: Results of Operations ...
key Results of Operations, input token length: 2099, output token length: 406
num_input_tokens: 2099, num_output_tokens: 278, total: 2377
working on 1013462: Revenues ...
key Revenues, input token length: 1387, output token length: 268
num_input_tokens: 1387, num_output_tokens: 212, total: 1599
working on 1013462: Gross Profit Margin ...
working on 1013462: Operating Income ...
key Operating Income, input token length: 484, output token length: 94
num_input_tokens: 484, num_output_tokens: 152, total: 636
working on 1013462: Interest Expense ...
working on 1013462: Liquidity ...
key Liquidity, input token length: 1539, output token length: 298
num_input_tokens: 1539, num_output_tokens: 266, total: 1805
working on 1013462: Debt ...
key Debt, input token length: 604, out

46it [1:50:55, 144.69s/it]


In [ ]:
# load in GPT generated summaries after creating them
summary_list = glob.glob(os.path.join(summary_path,'*.txt'))
len(summary_list)

46

In [ ]:
summary_df = []

def get_id(x):
  x = re.sub("/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/working3/gpt/", "", x)
  x = x.split(".")[0]
  return x

for s in summary_list:
  with open(s, 'r') as f:
    tmp_summary = "\n".join(f.readlines())
    summary_df.append({"id": get_id(s), "gpt_summary": tmp_summary})

summary_df = pd.DataFrame(summary_df)
summary_df.head()

,id,gpt_summary
0,8670,Business Overview\n\nADP is a global provider ...
1,50471,Business Overview\n\nThe company is a SaaS pro...
2,78749,Business Overview\n\nAgilysys is a hospitality...
3,317788,"Business Overview\n\nDigital Turbine, Inc. all..."
4,320340,Business Overview\n\nThe company provides tech...


In [ ]:
df = pd.merge(df, summary_df, how = "left", on = "id")
df.head()

,id,label_length,label,kept_report_length,report,has_label,gpt_summary
0,8670,1142,"AUTOMATIC DATA PROCESSING, INC. (“ADPI”) Auto...",3770,{'Business Overview': 'highlights from the yea...,True,Business Overview\n\nADP is a global provider ...
1,50471,907,"Park City Group, Inc. (“PCGI”) The Company is ...",1599,{'Business Overview': 'the company is a saas p...,True,Business Overview\n\nThe company is a SaaS pro...
2,78749,756,"AGILYSYS, Inc. (“AI”) Agilysys has been a lead...",3348,{'Business Overview': ' agilysys has been a le...,True,Business Overview\n\nAgilysys is a hospitality...
3,317788,927,"Digital Turbine, Inc. (“DTI”) Digital Turbine,...",6580,{'Business Overview': 'we allocate the fair va...,True,"Business Overview\n\nDigital Turbine, Inc. all..."
4,320340,933,Intelligent Systems Corporation (“ISC”) ISC’s...,1757,{'Business Overview': 'our consolidated operat...,True,Business Overview\n\nThe company provides tech...


In [ ]:
def calculate_metrics(ref_sentences, cand_sentences):
    # Load the models for BERTScore

    # Calculate the ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(cand_sentences, ref_sentences, avg=True)

    # Calculate the BERTScore
    bertscore = get_bert_score(cand_sentences.tolist(), ref_sentences.tolist())

    #return rouge_scores[0]['rouge-1']['f'],rouge_scores[0]['rouge-2']['f'],rouge_scores[0]['rouge-l']['f'],bertscore
    return rouge_scores['rouge-1']['f'], rouge_scores['rouge-2']['f'], rouge_scores['rouge-l']['f'], bertscore

def get_bert_score(cands, refs):
    assert len(cands) == len(refs)
    P, R, F1 = score(cands, refs, lang='en')
    P = torch.mean(P, dim=0).item()
    R = torch.mean(R, dim=0).item()
    F1 = torch.mean(F1, dim=0).item()
    return F1

In [ ]:
calculate_metrics(df['gpt_summary'], df['label'])

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(0.4905263727798257,
 0.27665308574061515,
 0.46643910380809533,
 0.85709148645401)

In [ ]:
df.to_pickle(os.path.join(summary_path, "gpt_summaries_df.pkl"))